In [ ]:
import numpy as np
from typing import List, Optional, Dict, Set, Callable, Any, Literal
from joblib import Memory, Parallel, delayed
import tslearn
import tslearn.metrics
from tslearn.datasets import UCR_UEA_datasets

from experiments.cross_validation import cv_tslearn, print_cv_results
from experiments.eval_on_test import validate_tslearn, print_test_results
from experiments.utils import join_dicts_from_pickle_paths, save_to_pickle

# Enumerate all tslearn datasets

In [ ]:
# _datasets = [
#             'ArticularyWordRecognition', 
#             'BasicMotions', 
#             'Cricket',
#             #'ERing',
#             'Libras', 
#             'NATOPS', 
#             'RacketSports',     
#             'FingerMovements',
#             'Heartbeat',
#             'SelfRegulationSCP1', 
#             'UWaveGestureLibrary'
#             ]

# import tslearn
# UCR_UEA_datasets = tslearn.datasets.UCR_UEA_datasets()

# for dataset_name in UCR_UEA_datasets.list_multivariate_datasets():
# #for dataset_name in _datasets:
#     print("Dataset:", dataset_name)
#     dataset = UCR_UEA_datasets.load_dataset(dataset_name)
#     if dataset[0] is not None:
#         X_train, y_train, X_test, y_test = dataset
#         num_classes = len(np.unique(y_train))
#         N_train, T, d = X_train.shape
#         N_test, _, _  = X_test.shape
        
#         print("Number of Classes:", num_classes)
#         print("Dimension of path:", d)
#         print("Length:", T)
#         print("Train Size, Test Size", N_train, N_test)
#         print()
#     else:
#         print("No dataset found")
#         print()

#yes
# Dataset: ArticularyWordRecognition
# Number of Classes: 25
# Dimension of path: 9
# Length: 144
# Train Size, Test Size 275 300

# Dataset: AtrialFibrillation
# No dataset found

#yes
# Dataset: BasicMotions
# Number of Classes: 4
# Dimension of path: 6
# Length: 100
# Train Size, Test Size 40 40

# Dataset: CharacterTrajectories
# No dataset found

#yes
# Dataset: Cricket
# Number of Classes: 12
# Dimension of path: 6
# Length: 1197
# Train Size, Test Size 108 72

# Dataset: DuckDuckGeese
# No dataset found

# Dataset: EigenWorms
# Number of Classes: 5
# Dimension of path: 6
# Length: 17984
# Train Size, Test Size 128 131

#why not
# Dataset: Epilepsy
# Number of Classes: 4
# Dimension of path: 3
# Length: 206
# Train Size, Test Size 137 138

#longLength
# Dataset: EthanolConcentration
# Number of Classes: 4
# Dimension of path: 3
# Length: 1751
# Train Size, Test Size 261 263

# Dataset: ERing
# No dataset found

#big
# Dataset: FaceDetection
# Number of Classes: 2
# Dimension of path: 144
# Length: 62
# Train Size, Test Size 5890 3524

#yes
# Dataset: FingerMovements
# Number of Classes: 2
# Dimension of path: 28
# Length: 50
# Train Size, Test Size 316 100

#why not, maybe big length
# Dataset: HandMovementDirection
# Number of Classes: 4
# Dimension of path: 10
# Length: 400
# Train Size, Test Size 160 74

#smallTrain
# Dataset: Handwriting
# Number of Classes: 26
# Dimension of path: 3
# Length: 152
# Train Size, Test Size 150 850

#yes
# Dataset: Heartbeat
# Number of Classes: 2
# Dimension of path: 61
# Length: 405
# Train Size, Test Size 204 205

#big
# Dataset: InsectWingbeat
# Number of Classes: 10
# Dimension of path: 200
# Length: 22
# Train Size, Test Size 25000 25000

# Dataset: JapaneseVowels
# No dataset found

#yes
# Dataset: Libras
# Number of Classes: 15
# Dimension of path: 2
# Length: 45
# Train Size, Test Size 180 180

#TODO I SHOULD INCLUDE
# Dataset: LSST
# Number of Classes: 14
# Dimension of path: 6
# Length: 36
# Train Size, Test Size 2459 2466

#length
# Dataset: MotorImagery
# Number of Classes: 2
# Dimension of path: 64
# Length: 3000
# Train Size, Test Size 278 100

#yes
# Dataset: NATOPS
# Number of Classes: 6
# Dimension of path: 24
# Length: 51
# Train Size, Test Size 180 180

#yes
# Dataset: PenDigits
# Number of Classes: 10
# Dimension of path: 2
# Length: 8
# Train Size, Test Size 7494 3498

#TODO SHOULD INCLUDE highDim
# Dataset: PEMS-SF
# Number of Classes: 7
# Dimension of path: 963
# Length: 144
# Train Size, Test Size 267 173

#NO, dim=1, big length, large num classes
# Dataset: Phoneme
# Number of Classes: 39
# Dimension of path: 1
# Length: 1024
# Train Size, Test Size 214 1896

#yes
# Dataset: RacketSports
# Number of Classes: 4
# Dimension of path: 6
# Length: 30
# Train Size, Test Size 151 152

#yes
# Dataset: SelfRegulationSCP1
# Number of Classes: 2
# Dimension of path: 6
# Length: 896
# Train Size, Test Size 268 293

# Dataset: SelfRegulationSCP2
# Number of Classes: 2
# Dimension of path: 7
# Length: 1152
# Train Size, Test Size 200 180

# Dataset: SpokenArabicDigits
# No dataset found

#NO, long, also very small set
# Dataset: StandWalkJump
# Number of Classes: 3
# Dimension of path: 4
# Length: 2500
# Train Size, Test Size 12 15

#yes
# Dataset: UWaveGestureLibrary
# Number of Classes: 8
# Dimension of path: 3
# Length: 315
# Train Size, Test Size 120 320


# Cross Validation on Train

In [ ]:
cv_results = cv_tslearn(
    dataset_names = [
        'Epilepsy',                    # N_corpus = 34      #I should probably further limit this to >100
        # 'EthanolConcentration',        # N_corpus = 65
        # 'FingerMovements',             # N_corpus = 158
        # 'HandMovementDirection',       # N_corpus = 40
        # 'Heartbeat',                   # N_corpus = 102
        # 'LSST',                        # N_corpus = 176
        # 'MotorImagery',                # N_corpus = 139
        # 'NATOPS',                      # N_corpus = 30
        # 'PenDigits',                   # N_corpus = 749
        # 'PEMS-SF',                     # N_corpus = 38
        # 'PhonemeSpectra',              # N_corpus = 85
        # 'RacketSports',                # N_corpus = 38
        # 'SelfRegulationSCP1',          # N_corpus = 134
        ],
    kernel_names = [
        "linear",
        "rbf",
        "poly",
        "integral rbf",
        "integral poly",
        "truncated sig",
        #"truncated sig rbf",
        #"signature pde rbf",
        #"gak",
        ],
        k=5,
        n_repeats=1,
        n_jobs_repeats=1,
        n_jobs_gram=1,
        verbose=False,
        )

##### Print CV results

In [ ]:
print_cv_results(cv_results)

In [ ]:
# Cross Validation Results             cv_testing.pkl
# Number of Classes: 4
# Dimension of path: 3
# Length: 206
# Train: 137
# Test: N/A

# conf_results

# linear
# final_score_avgs 1.1737691775296382
# params_score_avgs [1.174]
# thresh_score_avgs [0.795 0.988 0.995 1.009 1.043 1.054 0.979 0.981 0.953 0.941 0.908 0.875
#  0.841 0.833 0.794 0.798 0.763 0.717 0.73  0.699 0.694 0.717 0.741 0.625]
# EPILEPSY
# {'threshold': 2}
# RUNNING
# {'threshold': 3}
# SAWING
# {'threshold': 2}
# WALKING
# {'threshold': 10}

# rbf
# final_score_avgs 1.4216664612723662
# params_score_avgs [1.174 1.174 1.194 1.319 1.317]
# thresh_score_avgs [1.024 1.141 1.06  1.111 1.158 1.171 1.144 1.149 1.07  1.041 1.005 1.022
#  1.046 1.042 1.024 1.009 0.973 0.992 0.986 0.986 0.97  0.963 1.18  0.826]
# EPILEPSY
# {'sigma': 0.001, 'threshold': 2}
# RUNNING
# {'sigma': 1.0, 'threshold': 27}
# SAWING
# {'sigma': 10.0, 'threshold': 23}
# WALKING
# {'sigma': 1.0, 'threshold': 10}

# poly
# final_score_avgs 1.1743328857450155
# params_score_avgs [1.17  1.163 1.155 1.158]
# thresh_score_avgs [0.849 0.993 1.002 1.012 1.068 1.081 1.012 0.989 0.957 0.94  0.914 0.88
#  0.849 0.859 0.824 0.809 0.782 0.746 0.744 0.72  0.698 0.728 0.763 0.614]
# EPILEPSY
# {'p': 2, 'threshold': 2}
# RUNNING
# {'p': 2, 'threshold': 3}
# SAWING
# {'p': 5, 'threshold': 2}
# WALKING
# {'p': 2, 'threshold': 8}

# integral rbf
# final_score_avgs 1.3764592979160106
# params_score_avgs [1.18  1.18  1.187 1.292 1.248]
# thresh_score_avgs [0.981 1.133 1.153 1.116 1.126 1.089 1.078 1.037 1.071 1.038 1.009 0.987
#  0.977 0.999 0.997 0.986 0.959 0.931 0.918 0.905 0.898 0.932 1.177 0.766]
# EPILEPSY
# {'sigma': 0.001, 'threshold': 2}
# RUNNING
# {'sigma': 1.0, 'threshold': 27}
# SAWING
# {'sigma': 10.0, 'threshold': 23}
# WALKING
# {'sigma': 1.0, 'threshold': 13}

# integral poly
# final_score_avgs 1.325727324660064
# params_score_avgs [1.284 1.259 1.224 1.163]
# thresh_score_avgs [1.042 1.175 1.236 1.21  1.149 1.114 1.053 1.032 1.015 0.996 1.022 1.012
#  1.018 0.977 0.962 0.958 0.961 0.963 0.939 0.912 0.908 0.979 1.076 0.782]
# EPILEPSY
# {'p': 3, 'threshold': 2}
# RUNNING
# {'p': 2, 'threshold': 3}
# SAWING
# {'p': 3, 'threshold': 4}
# WALKING
# {'p': 2, 'threshold': 18}

# truncated sig
# final_score_avgs 1.3093519407707075
# params_score_avgs [1.309]
# thresh_score_avgs [1.074 1.06  1.063 0.972 1.087 1.076 1.055 1.049 1.041 1.059 1.046 1.087
#  1.088 1.046 1.123 0.983 0.971 0.986 0.979 0.976 0.989 0.998 0.99  0.712]
# orders_score_avgs [0.95  1.278 1.194 1.087 1.035 1.074 1.026 1.056 1.068 1.053]
# EPILEPSY
# {'threshold': 15, 'order': 2}
# RUNNING
# {'threshold': 2, 'order': 1}
# SAWING
# {'threshold': 13, 'order': 2}
# WALKING
# {'threshold': 16, 'order': 8}

# mahal_results

# linear
# final_score_avgs 1.0177805635643282
# params_score_avgs [1.018]
# thresh_score_avgs [0.708 0.931 0.96  0.93  0.908 0.858 0.81  0.769 0.735 0.711 0.685 0.675
#  0.659 0.644 0.637 0.627 0.612 0.609 0.603 0.594 0.587 0.562 0.59  0.472]
# EPILEPSY
# {'threshold': 3}
# RUNNING
# {'threshold': 1}
# SAWING
# {'threshold': 2}
# WALKING
# {'threshold': 28}

# rbf
# final_score_avgs 1.2733981379422619
# params_score_avgs [1.018 1.011 1.008 0.964 0.992]
# thresh_score_avgs [0.986 1.186 1.194 1.175 1.169 1.128 1.093 1.056 1.039 1.039 1.028 1.024
#  1.02  1.016 1.019 1.017 1.007 0.999 0.992 0.972 0.989 0.954 0.95  0.835]
# EPILEPSY
# {'sigma': 0.001, 'threshold': 3}
# RUNNING
# {'sigma': 10.0, 'threshold': 24}
# SAWING
# {'sigma': 0.001, 'threshold': 2}
# WALKING
# {'sigma': 0.001, 'threshold': 28}

# poly
# final_score_avgs 1.0456643672986525
# params_score_avgs [1.028 1.028 1.034 1.045]
# thresh_score_avgs [0.716 0.939 0.968 0.94  0.921 0.872 0.818 0.771 0.746 0.726 0.701 0.677
#  0.665 0.644 0.651 0.642 0.62  0.621 0.616 0.609 0.585 0.598 0.619 0.49 ]
# EPILEPSY
# {'p': 2, 'threshold': 3}
# RUNNING
# {'p': 5, 'threshold': 1}
# SAWING
# {'p': 5, 'threshold': 2}
# WALKING
# {'p': 5, 'threshold': 27}

# integral rbf
# final_score_avgs 1.2395262664840416
# params_score_avgs [1.015 1.008 1.008 0.937 0.811]
# thresh_score_avgs [0.848 1.14  1.197 1.187 1.175 1.136 1.095 1.039 1.02  0.999 0.959 0.951
#  0.955 0.933 0.937 0.936 0.92  0.906 0.907 0.904 0.917 0.894 0.908 0.777]
# EPILEPSY
# {'sigma': 0.001, 'threshold': 3}
# RUNNING
# {'sigma': 10.0, 'threshold': 4}
# SAWING
# {'sigma': 0.001, 'threshold': 2}
# WALKING
# {'sigma': 0.001, 'threshold': 28}

# integral poly
# final_score_avgs 0.9708220351081185
# params_score_avgs [0.971 0.913 0.891 0.886]
# thresh_score_avgs [0.847 0.906 0.887 0.885 0.896 0.872 0.868 0.843 0.848 0.826 0.813 0.8
#  0.804 0.794 0.778 0.744 0.755 0.746 0.732 0.727 0.712 0.722 0.747 0.559]
# EPILEPSY
# {'p': 2, 'threshold': 6}
# RUNNING
# {'p': 2, 'threshold': 1}
# SAWING
# {'p': 2, 'threshold': 2}
# WALKING
# {'p': 2, 'threshold': 27}

# truncated sig
# final_score_avgs 1.4164783869986826
# params_score_avgs [1.416]
# thresh_score_avgs [1.219 1.228 1.245 1.142 1.067 1.069 1.028 0.985 0.967 0.966 0.984 0.979
#  1.033 0.965 0.988 0.918 0.908 0.929 0.938 0.942 0.963 0.968 0.975 0.712]
# orders_score_avgs [0.933 1.067 1.03  1.279 1.176 1.151 1.103 1.246 1.186 1.214]
# EPILEPSY
# {'threshold': 15, 'order': 2}
# RUNNING
# {'threshold': 1, 'order': 8}
# SAWING
# {'threshold': 3, 'order': 6}
# WALKING
# {'threshold': 28, 'order': 3}

# End dataset 

In [ ]:
# from experiments.utils import save_to_pickle
# save_to_pickle(cv_results, "cv_testing.pkl")


from experiments.utils import load_from_pickle
cv_results = load_from_pickle("Data/cv_Epilepsy.pkl")
#cv_results = load_from_pickle("cv_testing.pkl")
del cv_results["Epilepsy"]["conf_results"]["gak"]
del cv_results["Epilepsy"]["conf_results"]["truncated sig rbf"]
del cv_results["Epilepsy"]["conf_results"]["signature pde rbf"]
del cv_results["Epilepsy"]["mahal_results"]["gak"]
del cv_results["Epilepsy"]["mahal_results"]["truncated sig rbf"]
del cv_results["Epilepsy"]["mahal_results"]["signature pde rbf"]

# Validate on Test

In [ ]:
test_results = validate_tslearn(cv_results, n_jobs=4, verbose=False)

##### Print test results

In [ ]:
print_test_results(test_results)

In [ ]:
# Test Results       of cv_Epilepsy.pkl  ---- WRONG????

# Dataset: Epilepsy
# Number of Classes: 4
# Dimension of path: 3
# Length: 206
# Train: 137
# Test: 138

# Kernel: linear
# Conformance AUC: 0.605
# Conformance PR AUC: 0.388

# Kernel: rbf
# Conformance AUC: 0.622
# Conformance PR AUC: 0.411

# Kernel: poly
# Conformance AUC: 0.621
# Conformance PR AUC: 0.398

# Kernel: truncated sig
# Conformance AUC: 0.626
# Conformance PR AUC: 0.367

# Kernel: integral rbf
# Conformance AUC: 0.628
# Conformance PR AUC: 0.428

# Kernel: integral poly
# Conformance AUC: 0.651
# Conformance PR AUC: 0.412

# Kernel: linear
# Mahalanobis AUC: 0.465
# Mahalanobis PR AUC: 0.307

# Kernel: rbf
# Mahalanobis AUC: 0.588
# Mahalanobis PR AUC: 0.368

# Kernel: poly
# Mahalanobis AUC: 0.423
# Mahalanobis PR AUC: 0.275

# Kernel: truncated sig
# Mahalanobis AUC: 0.651
# Mahalanobis PR AUC: 0.421

# Kernel: integral rbf
# Mahalanobis AUC: 0.576
# Mahalanobis PR AUC: 0.342

# Kernel: integral poly
# Mahalanobis AUC: 0.464
# Mahalanobis PR AUC: 0.281

# End Dataset




In [ ]:
# Test Results   first pool, then z

# Dataset: Epilepsy
# Number of Classes: 4
# Dimension of path: 3
# Length: 206
# Train: 137
# Test: 138

# Kernel: linear
# Conformance AUC: 0.618
# Conformance PR AUC: 0.425

# Kernel: rbf
# Conformance AUC: 0.612
# Conformance PR AUC: 0.439

# Kernel: poly
# Conformance AUC: 0.67
# Conformance PR AUC: 0.468

# Kernel: integral rbf
# Conformance AUC: 0.526
# Conformance PR AUC: 0.371

# Kernel: integral poly
# Conformance AUC: 0.687
# Conformance PR AUC: 0.423

# Kernel: truncated sig
# Conformance AUC: 0.628
# Conformance PR AUC: 0.368

# Kernel: linear
# Mahalanobis AUC: 0.516
# Mahalanobis PR AUC: 0.341

# Kernel: rbf
# Mahalanobis AUC: 0.712
# Mahalanobis PR AUC: 0.47

# Kernel: poly
# Mahalanobis AUC: 0.51
# Mahalanobis PR AUC: 0.324

# Kernel: integral rbf
# Mahalanobis AUC: 0.692
# Mahalanobis PR AUC: 0.449

# Kernel: integral poly
# Mahalanobis AUC: 0.503
# Mahalanobis PR AUC: 0.284

# Kernel: truncated sig
# Mahalanobis AUC: 0.712
# Mahalanobis PR AUC: 0.519

# End Dataset

In [ ]:
# Test Results    first pool, then z, then time augment

# Dataset: Epilepsy
# Number of Classes: 4
# Dimension of path: 3
# Length: 206
# Train: 137
# Test: 138

# Kernel: linear
# Conformance AUC: 0.566
# Conformance PR AUC: 0.364

# Kernel: rbf
# Conformance AUC: 0.487
# Conformance PR AUC: 0.341

# Kernel: poly
# Conformance AUC: 0.579
# Conformance PR AUC: 0.374

# Kernel: integral rbf
# Conformance AUC: 0.469
# Conformance PR AUC: 0.334

# Kernel: integral poly
# Conformance AUC: 0.697
# Conformance PR AUC: 0.49

# Kernel: truncated sig
# Conformance AUC: 0.703
# Conformance PR AUC: 0.45

# Kernel: linear
# Mahalanobis AUC: 0.496
# Mahalanobis PR AUC: 0.314

# Kernel: rbf
# Mahalanobis AUC: 0.689
# Mahalanobis PR AUC: 0.431

# Kernel: poly
# Mahalanobis AUC: 0.502
# Mahalanobis PR AUC: 0.317

# Kernel: integral rbf
# Mahalanobis AUC: 0.682
# Mahalanobis PR AUC: 0.426

# Kernel: integral poly
# Mahalanobis AUC: 0.518
# Mahalanobis PR AUC: 0.316

# Kernel: truncated sig
# Mahalanobis AUC: 0.696
# Mahalanobis PR AUC: 0.516

# End Dataset

# Read CV data from file and print results

In [ ]:
# Load the cross validation results
cv_results = join_dicts_from_pickle_paths(
    [
    "Data/cv_Epilepsy.pkl",
    #"Data/cv_EthanolConcentration.pkl",
    #"Data/cv_FingerMovements.pkl",
    #"Data/cv_HandMovementDirection.pkl",
    #"Data/cv_Heartbeat.pkl",
    #"Data/cv_LSST.pkl",
    #"Data/cv_MotorImagery.pkl",
    #"Data/cv_NATOPS.pkl",
    #"Data/cv_PEMS-SF.pkl",
    #"Data/cv_PenDigits.pkl",
    #"Data/cv_PhonemeSpectra.pkl",
    #"Data/cv_RacketSports.pkl",
    #"Data/cv_SelfRegulationSCP1.pkl",
    ])
print_cv_results(cv_results)

In [ ]:
from experiments.experiment_code import print_dataset_stats
from experiments.utils import print_latex_results, join_dicts_from_pickle_paths

# test_results = join_dicts_from_pickle_paths([
#                                 "Data/results_shorts.pkl",
#                                 "Data/results_longs.pkl",
#                                              ])

test_results = {d:k for d,k in test_results.items() 
                # if d in ["EthanolConcentration",  #datasets with corpus size > 50
                #         "FingerMovements",
                #         "Heartbeat",
                #         "LSST",
                #         "MotorImagery",
                #         "PenDigits",
                #         "PhonemeSpectra",
                #         "SelfRegulationSCP1",]
                }
print_latex_results(test_results, 2)
print_latex_results(test_results, 3)